In [17]:
%load_ext autoreload
%reload_ext autoreload
%autoreload
from trackingutil import within_bound, get_bound, inframe_tracker

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
import unittest
import time
class TestUtilMethods(unittest.TestCase):

    def test_within_bound(self):
        self.assertTrue(within_bound(((1,2),(4,5)),((2,3),(4,4))))
        self.assertTrue(within_bound(((1,2),(4,5)),((1,3),(4,5))))
        self.assertFalse(within_bound(((1,2),(4,5)),((0,3),(4,5))))
        self.assertFalse(within_bound(((1,2),(4,5)),((1,3),(5,5))))
    
    def test_get_bound(self):
        self.assertEqual(((9, 9), (16,16)), get_bound(((10, 10), (15,15)), 0.1))
        self.assertEqual(((9, 9), (16,16)), get_bound(((10, 10), (15,15)), 0.2))
        self.assertEqual(((9, 9), (16,16)), get_bound(((10, 10), (15,15)), 0.4))
        self.assertEqual(((8, 8), (17,17)), get_bound(((10, 10), (15,15)), 0.8))
    def test_inframe_tracker(self):
        testTracker = inframe_tracker(0.5, 0.1)
        self.assertEqual(None,testTracker.bound)
        testTracker.track(((1,2),(3,4)))
        self.assertEqual(get_bound(((1,2),(3,4)), 0.1),testTracker.bound)
        testTracker.track(((-1,2),(3,4)))
        time.sleep(1)
        testTracker.track(None)
        self.assertEqual(2,len(testTracker.timesInframe))
        self.assertTrue(sum(testTracker.timesInframe)/len(testTracker.timesInframe) > 0.5 and sum(testTracker.timesInframe)/len(testTracker.timesInframe) < 1)
TestUtilMethods().test_within_bound()
TestUtilMethods().test_get_bound()
TestUtilMethods().test_inframe_tracker()